In [1]:
import os
import json

# Path to the folder containing all your detection JSON files
DETECTIONS_DIR = "/Users/rishitmahapatra/Documents/GitHub/commentary_generation/Datasets/Object_detection_frames_with_metadata"  

# Collect detections
detections_data = {}

# Loop over all JSON files
for file in sorted(os.listdir(DETECTIONS_DIR)):
    if file.endswith(".json"):
        with open(os.path.join(DETECTIONS_DIR, file), "r") as f:
            data = json.load(f)
            frame_name = data["frame"]
            detections_data[frame_name] = data["detections"]

print(f"✅ Loaded {len(detections_data)} frames with detections.")


✅ Loaded 2700 frames with detections.


In [2]:
import numpy as np

# How far (pixels) two detections can be considered the same object
DIST_THRESHOLD = 50  

# Store tracked objects per frame
tracked_detections = {}

# Running ID counter
next_id = 1

# Sort frames by name to process sequentially
sorted_frames = sorted(detections_data.keys())

# Loop over frames
for i, frame in enumerate(sorted_frames):
    detections = detections_data[frame]
    tracked_this_frame = []

    if i == 0:
        # First frame: assign IDs directly
        for det in detections:
            det["id"] = next_id
            next_id += 1
            tracked_this_frame.append(det)
    else:
        prev_frame = sorted_frames[i - 1]
        prev_detections = tracked_detections[prev_frame]

        # For each detection in this frame
        for det in detections:
            x1, y1, x2, y2 = det["bbox"]
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2

            # Find closest previous detection
            min_dist = float("inf")
            matched_id = None
            for prev_det in prev_detections:
                px1, py1, px2, py2 = prev_det["bbox"]
                pcx = (px1 + px2) / 2
                pcy = (py1 + py2) / 2
                dist = np.sqrt((cx - pcx) ** 2 + (cy - pcy) ** 2)

                if dist < min_dist:
                    min_dist = dist
                    matched_id = prev_det["id"]

            # Assign ID if close enough
            if min_dist < DIST_THRESHOLD:
                det["id"] = matched_id
            else:
                det["id"] = next_id
                next_id += 1

            tracked_this_frame.append(det)

    # Store for this frame
    tracked_detections[frame] = tracked_this_frame

print(f"✅ Tracking IDs assigned across {len(sorted_frames)} frames.")


✅ Tracking IDs assigned across 2700 frames.


In [7]:
event_log = []

# Tuning parameters
BALL_SPEED_SHOT = 30
BALL_SPEED_PASS = 10
NEAR_GOAL_LINE_X = 50
GOAL_AREA_Y_MIN = 100
GOAL_AREA_Y_MAX = 300
CLOSE_PLAYER_DISTANCE = 50
MAX_MISSING_FRAMES = 3

last_ball_position = None
ball_missing_count = 0

for frame_id, curr_frame in tracked_detections.items():
    # Find ball
    curr_ball = next((d for d in curr_frame if d["class"] == "sports ball"), None)

    # Ball detected this frame
    if curr_ball:
        bx, by = curr_ball["bbox"][:2]

        if last_ball_position:
            prev_x, prev_y = last_ball_position
            dx = bx - prev_x
            dy = by - prev_y
            dist = (dx**2 + dy**2)**0.5

            # Shot
            if dist > BALL_SPEED_SHOT and bx < NEAR_GOAL_LINE_X:
                event_log.append({
                    "frame": frame_id,
                    "event": "Shot",
                    "details": f"Fast ball toward goal, dx={dx:.1f}, dy={dy:.1f}"
                })

            # Goal
            if bx < NEAR_GOAL_LINE_X and GOAL_AREA_Y_MIN < by < GOAL_AREA_Y_MAX:
                event_log.append({
                    "frame": frame_id,
                    "event": "Goal",
                    "details": "Ball entered goal area."
                })

            # Pass
            elif BALL_SPEED_PASS < dist <= BALL_SPEED_SHOT:
                event_log.append({
                    "frame": frame_id,
                    "event": "Pass",
                    "details": f"Ball moved {dist:.1f}px"
                })

            # Dribble
            elif dist <= BALL_SPEED_PASS:
                event_log.append({
                    "frame": frame_id,
                    "event": "Dribble",
                    "details": "Slow ball movement"
                })

        last_ball_position = (bx, by)
        ball_missing_count = 0

    # Ball missing this frame
    else:
        ball_missing_count += 1

        # If ball was seen recently
        if last_ball_position and ball_missing_count <= MAX_MISSING_FRAMES:
            bx, by = last_ball_position
            if bx < NEAR_GOAL_LINE_X and GOAL_AREA_Y_MIN < by < GOAL_AREA_Y_MAX:
                event_log.append({
                    "frame": frame_id,
                    "event": "Goal (ball disappeared)",
                    "details": "Ball vanished in goal area."
                })
            elif bx < 100 and int(frame_id.split("_")[1].split(".")[0]) < 5:
                event_log.append({
                    "frame": frame_id,
                    "event": "Kickoff (ball disappeared)",
                    "details": "Early frame ball vanished near center."
                })
        else:
            last_ball_position = None

    # Check for player proximity
    for det in curr_frame:
        if det["class"] == "person":
            px, py = det["bbox"][:2]
            if curr_ball:
                dist = ((bx - px)**2 + (by - py)**2)**0.5
                if dist < CLOSE_PLAYER_DISTANCE:
                    event_log.append({
                        "frame": frame_id,
                        "event": "Player Near Ball",
                        "details": f"Distance {dist:.1f}px"
                    })

print(f"✅ Advanced event logic completed for {len(tracked_detections)} frames.")


✅ Advanced event logic completed for 2700 frames.


In [10]:
import json

OUTPUT_JSON = "/Users/rishitmahapatra/Documents/GitHub/commentary_generation/Datasets/Event_log_JSON/event_log.json"

with open(OUTPUT_JSON, "w") as f:
    json.dump(event_log, f, indent=2)

print(f"✅ Event log saved to {OUTPUT_JSON}")


✅ Event log saved to /Users/rishitmahapatra/Documents/GitHub/commentary_generation/Datasets/Event_log_JSON/event_log.json
